# Sentiment Analysis Notebook

## Dataset

The data given is in the form of a comma-separated values files with tweets and their corresponding sentiments. The training dataset is a csv file of type tweet_id,sentiment,tweet where the
tweet_id is a unique integer identifying the tweet, sentiment is either 1 (positive) or 0 (negative), and tweet is the tweet enclosed in "". Similarly, the test dataset is a csv file of type
tweet_id,tweet.

In [38]:
import pandas as pd
import sys
sys.path.insert(0, 'src')

In [39]:
dataset = pd.read_csv('dataset/train.csv', encoding = "ISO-8859-1")
dataset.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


## Preprocessing

### Preprocessing functions

In [40]:
import re
from nltk.stem.porter import PorterStemmer

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def preprocess_word(word):
    # remove punctation
    word = word.strip('\'"?!,.():;')
    # more than 3 letter repetition removed
    word = re.sub(r'(.)\1\1+', r'\1\1\1', word)
    # remove - & '
    word = word.strip('-&\'')
    return word

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def preprocess_tweet(tweet, use_stemmer = False):
    # convert tweet to lowercase
    tweet = tweet.lower()
    # replace urls with 'URL'
    tweet = re.sub(r'((www.[\S]+)|(https?://.[\S]+))', 'URL', tweet)
    # replace user mentions @user with 'USER_MENTION'
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # replace #hashtag with hastag
    tweet = re.sub(r'#(\S+)', r' \1', tweet)
    # remove retweet RT
    tweet = re.sub(r'\brt\b', '', tweet)
    # replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # remove space, " and ' 
    tweet.strip('" \'')
    # handle emojis. Use only EMO_POS and EMO_NEG
    tweet = handle_emojis(tweet)
    # replace multiple spaces with only one space
    tweet = re.sub(r'\s+', ' ', tweet)
    # preprocess words
    words = tweet.split()

    processed_words = []
    porter_stemmer = PorterStemmer()
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                # use stemmer
                word = str(porter_stemmer.stem(word))
            processed_words.append(word)
    return ' '.join(processed_words)

In [41]:
tweet = "CIAO COME vaaaaa? :) :( https://tartarus.org/martin/PorterStemmer/  bella @ivocerti    #sentimentanalysis"
print(preprocess_tweet(tweet))
print(preprocess_tweet(dataset.loc[1]['SentimentText']))

ciao come vaaa EMO_POS EMO_NEG URL bella USER_MENTION sentimentanalysis
i missed the new moon trailer


### Preprocess the whole dataset

In [42]:
preprocessed_dataset = dataset
for i in range(dataset.size):
    preprocessed_dataset.at[i, 'SentimentText'] = preprocess_tweet(dataset.loc[i]['SentimentText'])

KeyError: 'the label [99989] is not in the [index]'

In [51]:
dataset.loc[99988]['SentimentText']

'USER_MENTION haha yes you do'

## Vectorization